# Screener 02

https://readmedium.com/en/https:/wire.insiderfinance.io/identifying-breakout-stocks-with-python-a-data-driven-selection-guide-d8a3d1ba172e
https://github.com/shashankvemuri/Finance/blob/master/find_stocks/get_rsi_tickers.py

https://medium.com/@redeaddiscolll
https://medium.com/@py.chin315
https://github.com/peiyingchin/Medium/blob/main/Trading%20strategy%20with%2055%25%20win%20chance/Trading%20strategy%20with%2055%25%20win%20chance.ipynb

In [2]:
import pandas as pd

# Define the function to load data and select breakout stock candidates
def select_breakout_candidates(csv_path):
    # Load the dataset
    stock_data = pd.read_csv(csv_path)
    
    # Preprocessing and filtering based on the provided criteria
    stock_data_filtered = stock_data.dropna(subset=['Target Price', 'P/E'])
    stock_data_filtered = stock_data_filtered[stock_data_filtered['Price'] < stock_data_filtered['Target Price']]
    
    # Calculate additional metrics
    stock_data_filtered['Price to Target %'] = ((stock_data_filtered['Target Price'] - stock_data_filtered['Price']) / stock_data_filtered['Price']) * 100
    stock_data_filtered['Change Percentile'] = stock_data_filtered['Change'].rank(pct=True)
    stock_data_filtered['Volume Percentile'] = stock_data_filtered['Volume'].rank(pct=True)
    stock_data_filtered['Price to Target Percentile'] = stock_data_filtered['Price to Target %'].rank(pct=True)
    stock_data_filtered['Composite Score'] = stock_data_filtered[['Change Percentile', 'Volume Percentile', 'Price to Target Percentile']].mean(axis=1)
    
    # Exclude overvalued stocks based on P/E ratio threshold
    pe_threshold = stock_data_filtered['P/E'].quantile(0.80)
    candidates = stock_data_filtered[stock_data_filtered['P/E'] <= pe_threshold]
    
    # Select top candidates based on composite score
    top_candidates = candidates.sort_values(by='Composite Score', ascending=False).head(8)
    
    # Return the selection with relevant information for the article
    return top_candidates[['Ticker', 'Company', 'Sector', 'Industry', 'Price', 'Volume', 'P/E', 'Target Price', 'Price to Target %', 'Composite Score']]

# Usage of the function with the path to the CSV file
csv_file_path = 'nifty50_symbols.txt'  # Replace with your actual file path
selected_stocks = select_breakout_candidates(csv_file_path)

# Display the selected stocks
selected_stocks

KeyError: ['Target Price', 'P/E']

## RSI

In [ ]:
# Import Dependencies
import datetime
from pandas_datareader import data as pdr
import sys
import os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
import ta_functions as ta
import tickers as ti

# Get dates for the past year
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

# Load list of S&P 500 tickers from tickers module
tickers = ti.tickers_sp500()

# Initialize lists for overbought and oversold tickers
oversold_tickers = []
overbought_tickers = []

# Retrieve adjusted close prices for the tickers
sp500_data = pdr.get_data_yahoo(tickers, start_date, end_date)['Adj Close']

# Analyze each ticker for RSI
for ticker in tickers:
    try:
        # Create a new DataFrame for the ticker
        data = sp500_data[[ticker]].copy()

        # Calculate the RSI for the ticker
        data["rsi"] = ta.RSI(data[ticker], timeperiod=14)

        # Calculate the mean of the last 14 RSI values
        mean_rsi = data["rsi"].tail(14).mean()

        # Print the RSI value
        print(f'{ticker} has an RSI value of {round(mean_rsi, 2)}')

        # Classify the ticker based on its RSI value
        if mean_rsi <= 30:
            oversold_tickers.append(ticker)
        elif mean_rsi >= 70:
            overbought_tickers.append(ticker)

    except Exception as e:
        print(f'Error processing {ticker}: {e}')

# Output the lists of oversold and overbought tickers
print(f'Oversold tickers: {oversold_tickers}')
print(f'Overbought tickers: {overbought_tickers}')